In [32]:
import inspect


class Ordered(object):

    # Default order value
    _order = 0

    def __init_subclass__(cls, *args, **kwargs) -> None:
        # Set class order value
        cls._order = kwargs.pop('order', cls._order)
        # Call super
        super().__init_subclass__(*args, **kwargs)

    @property
    def order(self):
        return self._order
    
    @order.setter
    def order(self, value):
        if isinstance(value, int):
            self._order = value
        else:
            raise TypeError("Value must be of type int")


class Container(Ordered):
    
    def __init__(self, *args, **kwargs) -> None:
        # Delegate to super
        super().__init__(*args, **kwargs)
        # Initialize members
        self._init_members()

    def _init_members(self):
        # Initialize inner classes as members
        members = {}
        for k, v in self.__class__.__dict__.items():
            if inspect.isclass(v):
                if issubclass(v, Ordered):
                    members[k] = v()
        # Sort members by order
        members = dict(sorted(members.items(), key=lambda x: x[1].order))
        self._members = members

    @property
    def members(self):
        return self._members

class Processor(Ordered):
    
    def __init__(self, *args, **kwargs) -> None:
        # Delegate to super
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        pass

In [44]:
cpm = CPM()
cpm.members

{'AFs': <__main__.CPM.AFs at 0x7f4a98621e10>,
 'SPFs': <__main__.CPM.SPFs at 0x7f4a986209a0>}

In [43]:
import math

class CPM(Container):
    """
    Crash prediction model for rural two-lane roads based on HSM chapter 10.
    """

    class SPFs(Container, order=0):
        """
        Safety performance functions for rural two-lane roads.
        """

        #@Processor(pre=Processor('spf.json'))
        def n_kabco(aadt, length):
            """
            Based on HSM Equation 10-7.
            """
            # Compute number of crashes
            n = aadt * length * 365 * 10E-6 * math.exp(-0.312)
            return n

        #@Processor()
        def overdispersion(length):
            # Compute overdispersion
            k = 0.236 / length
            return k

    class AFs(Container, order=1):
        """
        Adjustment factors for rural two-lane roads.
        """
        pass